In [1]:
mod_input=True
mod_topo=True
leak_rate = 0.2
gradient= 0.00075# 0.0005
gain = 1.75
radical=False
radical_bis=False
grad2=1.25
breadth=600
expon=3

remove_stop_words = False
do_shuffle = True


if do_shuffle:
    intact_status_txt = "shuffled"
else:
    intact_status_txt = "intact"

print(intact_status_txt)
# baseline with slow leak rate

shuffled


In [2]:
import os
create_directories = False
if create_directories:
    os.mkdir("multi_syn_txt_data")               #main dir

    for nar_text in range(0,8):
        os.mkdir("multi_syn_txt_data/syn_nar_%d" % (nar_text))               #main dir
        os.mkdir("multi_syn_txt_data/syn_nar_%d/intact" % (nar_text))        #intact subdir
        os.mkdir("multi_syn_txt_data/syn_nar_%d/intact/data" % (nar_text))
        os.mkdir("multi_syn_txt_data/syn_nar_%d/intact/figures" % (nar_text))
        os.mkdir("multi_syn_txt_data/syn_nar_%d/shuffled" % (nar_text))        #intact subdir
        os.mkdir("multi_syn_txt_data/syn_nar_%d/shuffled/data" % (nar_text))
        os.mkdir("multi_syn_txt_data/syn_nar_%d/shuffled/figures" % (nar_text))
   


In [3]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pylab
import statsmodels.api as sm

import seaborn as sns

import csv

import scipy
from scipy import stats
from scipy.ndimage import gaussian_filter1d

import random


In [4]:
#import cupy as cp
from easyesn.optimizers import GradientOptimizer
from easyesn import PredictionESN
from easyesn.optimizers import GridSearchOptimizer
from easyesn import helper as hlp
import numpy as np
from wikipedia2vec import Wikipedia2Vec



vectorDim = 100

numNode = 1000

#inputDataTraining = np.load('./trainingData_averaging/inputDataTraining_4k_average.npy')
#outputDataTraining = np.load('./trainingData_averaging/outputDataTraining_4k_average.npy')
#print(len(inputDataTraining))
#print(len(outputDataTraining))

Using Numpy backend.


In [5]:
#reservoirStatesBuffer[:,:resn]

#in_small = inputDataTraining[:10000]
#out_small = outputDataTraining[:10000]


#np.save('in_small.npy', in_small)
#np.save('out_small.npy', out_small)

in_small = np.load('C:/Users/PeterDell/Google Drive/GoogleWIP/People/Uchida/PaperMethods/UchidaPrograms/in_small.npy')
out_small = np.load('C:/Users/PeterDell/Google Drive/GoogleWIP/People/Uchida/PaperMethods/UchidaPrograms/out_small.npy')

print(len(in_small))
print(len(out_small))

10000
10000


In [6]:
temp_W=np.zeros([1000,1000])

In [7]:
for nar_text in range(0,8):
    current_file = "syn_nar_texts/syn_narr_%d.npy" % (nar_text)
    print(current_file)
    inputDataTesting=np.load(current_file)
    print(inputDataTesting.shape)
    current_file = "syn_nar_texts/syn_narr_gt_%d.npy" % (nar_text)
    ground_truth_sums = np.load(current_file)
    print(ground_truth_sums)

syn_nar_texts/syn_narr_0.npy
(2029, 100)
[  87  222  252  396  524  646  687  712  805  907 1009 1178 1292 1352
 1519 1561 1663 1817 1925]
syn_nar_texts/syn_narr_1.npy
(1834, 100)
[ 124  266  312  456  489  565  693  734  844  912 1069 1213 1280 1444
 1470 1495 1584 1666 1712]
syn_nar_texts/syn_narr_2.npy
(2087, 100)
[  43   78  223  379  542  674  785  944  993 1137 1307 1359 1424 1531
 1625 1748 1779 1828 1978]
syn_nar_texts/syn_narr_3.npy
(2068, 100)
[ 149  225  260  341  411  566  647  704  807  874 1001 1175 1285 1404
 1547 1624 1745 1879 1980]
syn_nar_texts/syn_narr_4.npy
(1813, 100)
[  88  114  200  295  422  507  542  665  778  885 1054 1181 1259 1349
 1437 1532 1601 1640 1750]
syn_nar_texts/syn_narr_5.npy
(2227, 100)
[ 161  284  457  530  684  821  880  967 1002 1174 1295 1452 1497 1547
 1710 1884 2052 2120 2200]
syn_nar_texts/syn_narr_6.npy
(1753, 100)
[  82  156  273  345  445  484  617  666  733  830  884 1052 1095 1133
 1222 1271 1342 1516 1591]
syn_nar_texts/syn_narr_7.np

In [8]:
for nar_text in range(5,8): #8
    current_file = "syn_nar_texts/syn_narr_%d.npy" % (nar_text)
    print(current_file)
    inputDataTesting=np.load(current_file)
    print(inputDataTesting.shape)
    current_file = "syn_nar_texts/syn_narr_gt_%d.npy" % (nar_text)
    ground_truth_sums = np.load(current_file)
    print(ground_truth_sums)    
###################################################################
    # see if we are shuffling
    if do_shuffle:
        random.shuffle(inputDataTesting)
        random.shuffle(inputDataTesting)
        random.shuffle(inputDataTesting)

##########   now run the reservoirs for each of the input texts

    for res_seed in range(40): #40
    # set the seed
        np.random.seed(res_seed)

        # set up the reservoir
        # leakingRate=0.2 changing too fast, mutliple with 0.05; changed
        esn = PredictionESN(n_input=vectorDim, n_output=vectorDim, n_reservoir=numNode, leakingRate=leak_rate, regressionParameters=[1e-2], solver="lsqr", feedback=False)

        # train the reervoir with a new seed
        esn.fit(in_small, out_small, transientTime="Auto", verbose=1)



        # modify the reservoir topology for input
        if mod_input:
            for i in range(700):
                for j in range(101):
                    esn._WInput[i+300,j] = 0
            esn._WInput=esn._WInput*5

        #radical _W modification
        if mod_topo:
            for i in range(1000):
                for j in range(1000):
                    temp_W[i,j] = esn._W[i,j]
            for i in range(1000):
                for j in range(1000):
                    esn._W[i,j]=0
            for i in range(esn._W.shape[0]):
                for j in range(esn._W.shape[1]):
                    if abs(i-j) <= breadth:
                        esn._W[i,j] = ((breadth-abs(i-j))/breadth)**expon*temp_W[i,j]* (1 + i * gradient)*gain
        if radical:
            for i in range(100,200):
                for j in range(800,900):
                    esn._W[j,i] = temp_W[j,i]*grad2

        if radical_bis:
            for i in range(100,200):
                for j in range(800,900):
                    esn._W[i,j] = temp_W[j,i]*grad2


        #run the reservoir
        reservoirStatesBuffer = np.empty((0,numNode))
        prediction,reservoirStatesBuffer  = esn.predict(inputDataTesting)
        reservoirStatesBuffer = reservoirStatesBuffer.T
        print(reservoirStatesBuffer.shape)
        reservoirStatesBuffer = reservoirStatesBuffer[:,101:]
        print(reservoirStatesBuffer.shape)
        print(prediction.shape)

        # set the similarity matrix
        #sim_matrix1 = np.empty([len(reservoirStatesBuffer),len(reservoirStatesBuffer)])

        print(len(reservoirStatesBuffer[1]))

        reservoirStatesBuffer1 = reservoirStatesBuffer

        #  multi_syn_txt_data\syn_nar_0\intact\data
        file_string = "multi_syn_txt_data/syn_nar_%d/%s/data/syn_nar_%d_%s_%d.npy" % (nar_text,intact_status_txt,nar_text,intact_status_txt,res_seed)
        print("saving file", file_string)
        np.save(file_string,reservoirStatesBuffer)

        #plot figure to verigy
        resn = 50
        begin=0
        end=2200
        rbegin = 900
        rend = 910
        fig= plt.figure(figsize=(10,12))

        fig.suptitle("syn_nar_%d_%s_seed_%d" % (nar_text,intact_status_txt,res_seed))

        plt.subplot(611)
        plt.plot( reservoirStatesBuffer1[:,100:105])
        plt.title('Hierarchical 100:105')

        plt.subplot(612)
        plt.plot( reservoirStatesBuffer1[:,300:305])
        plt.title('Hierarchical 300:305')

        plt.subplot(613)
        plt.plot( reservoirStatesBuffer1[:,500:505])
        plt.title('Hierarchical 500:505')

        plt.subplot(614)
        plt.plot( reservoirStatesBuffer1[:,700:705])
        plt.title('Hierarchical 700:705')

        plt.subplot(615)
        plt.plot( reservoirStatesBuffer1[:,900:905])
        plt.title('Hierarchical 900:905')

        plt.subplot(616)
        plt.plot( inputDataTesting[:,:])
        plt.title('Input')

        #fig.tight_layout()
        plt.savefig("multi_syn_txt_data/syn_nar_%d/%s/figures/syn_nar_%d_%s_%d.png" % (nar_text,intact_status_txt,nar_text,intact_status_txt,res_seed))

        fig= plt.figure(figsize=(10,8))
        sns.heatmap(np.corrcoef(reservoirStatesBuffer[:,:]),cmap='viridis')#sns.heatmap(sim_matrix, square=True)
        for i in ground_truth_sums:
            plt.plot(i, i,marker="o", markersize=8, markeredgecolor="red", markerfacecolor="red")        
        plt.savefig("multi_syn_txt_data/syn_nar_%d/%s/figures/syn_nar_%d_%s_cor_%d.png" % (nar_text,intact_status_txt,nar_text,intact_status_txt,res_seed))

        
        
        plt.close('all')    


syn_nar_texts/syn_narr_5.npy
(2227, 100)
[ 161  284  457  530  684  821  880  967 1002 1174 1295 1452 1497 1547
 1710 1884 2052 2120 2200]


100% (9904 of 9904) |####################| Elapsed Time: 0:00:05 Time:  0:00:05


(2227, 1101)
(2227, 1000)
(2227, 100)
1000
saving file multi_syn_txt_data/syn_nar_5/shuffled/data/syn_nar_5_shuffled_0.npy


100% (9906 of 9906) |####################| Elapsed Time: 0:00:06 Time:  0:00:06


(2227, 1101)
(2227, 1000)
(2227, 100)
1000
saving file multi_syn_txt_data/syn_nar_5/shuffled/data/syn_nar_5_shuffled_1.npy


...

(1774, 1101)
(1774, 1000)
(1774, 100)
1000
saving file multi_syn_txt_data/syn_nar_7/shuffled/data/syn_nar_7_shuffled_37.npy


100% (9905 of 9905) |####################| Elapsed Time: 0:00:11 Time:  0:00:11


(1774, 1101)
(1774, 1000)
(1774, 100)
1000
saving file multi_syn_txt_data/syn_nar_7/shuffled/data/syn_nar_7_shuffled_38.npy


100% (9902 of 9902) |####################| Elapsed Time: 0:00:10 Time:  0:00:10


(1774, 1101)
(1774, 1000)
(1774, 100)
1000
saving file multi_syn_txt_data/syn_nar_7/shuffled/data/syn_nar_7_shuffled_39.npy


In [9]:
break

SyntaxError: 'break' outside loop (<ipython-input-9-6aaf1f276005>, line 4)

In [ ]:
plt.imshow(esn._W,cmap='seismic')
plt.colorbar()

In [ ]:
plt.imshow(esn._WInput,cmap='seismic' )

In [ ]:
resn = 50
begin=0
end=680
rbegin = 900
rend = 910
fig= plt.figure(figsize=(10,12))

plt.subplot(511)
plt.plot( reservoirStatesBuffer1[begin:end,100:105])
plt.title('Hierarchical 100:105')

plt.subplot(512)
plt.plot( reservoirStatesBuffer1[begin:end,300:305])
plt.title('Hierarchical 300:305')

plt.subplot(513)
plt.plot( reservoirStatesBuffer1[begin:end,500:505])
plt.title('Hierarchical 500:505')

plt.subplot(514)
plt.plot( reservoirStatesBuffer1[begin:end,700:705])
plt.title('Hierarchical 700:705')

plt.subplot(515)
plt.plot( reservoirStatesBuffer1[begin:end,900:905])
plt.title('Hierarchical 900:905')


fig.tight_layout()

#plt.savefig('temporal_hierarchy.png')

In [ ]:
print(reservoirStatesBuffer1.shape)

In [ ]:
plt.plot( sm.tsa.stattools.ccf(reservoirStatesBuffer1[:,300], reservoirStatesBuffer1[:,350])[:20],linewidth=2 )
plt.plot( sm.tsa.stattools.ccf(reservoirStatesBuffer1[:,300], reservoirStatesBuffer1[:,450])[:20],linewidth=4 )
plt.plot( sm.tsa.stattools.ccf(reservoirStatesBuffer1[:,300], reservoirStatesBuffer1[:,550])[:20],linewidth=8 )

In [ ]:
reservoirStatesBuffer1.shape

In [ ]:
areas = 20
factor = int(numNode/areas)
roi_data = np.zeros([areas,reservoirStatesBuffer1.shape[0]])
print(roi_data.shape)
print(factor)

In [ ]:
for i in range(areas): # create 10 sub areas
    for j in range (reservoirStatesBuffer1.shape[0]): #for each timestep
        roi_data[i,j] = np.mean(reservoirStatesBuffer1[j,i*factor:(i+1)*factor])

In [ ]:
reservoirStatesBuffer1[j,i*factor:(i+1)*factor]

In [ ]:
plt.plot(roi_data.T)

plt.plot()

In [ ]:
plt.plot(roi_data[8:15,50:150].T)

plt.plot()

In [ ]:
source = 6
plt.plot( sm.tsa.stattools.ccf(roi_data[source,50:150].T, roi_data[8,50:150].T)[:10],linewidth=1 )
plt.plot( sm.tsa.stattools.ccf(roi_data[source,50:150].T, roi_data[10,50:150].T)[:10],linewidth=2 )
plt.plot( sm.tsa.stattools.ccf(roi_data[source,50:150].T, roi_data[12,50:150].T)[:10],linewidth=3 )
plt.plot( sm.tsa.stattools.ccf(roi_data[source,50:150].T, roi_data[14,50:150].T)[:10],linewidth=4 )
plt.plot( sm.tsa.stattools.ccf(roi_data[source,50:150].T, roi_data[16,50:150].T)[:10],linewidth=5 )
plt.plot( sm.tsa.stattools.ccf(roi_data[source,50:150].T, roi_data[18,50:150].T)[:10],linewidth=6 )
#plt.plot( sm.tsa.stattools.ccf(roi_data[source,50:150].T, roi_data[19,50:150].T)[:10],linewidth=7 )

In [ ]:
fig, [ax1, ax2] = plt.subplots(2, 1, sharex=True)
ax1.xcorr(reservoirStatesBuffer1[700,50:150].T, reservoirStatesBuffer1[800,50:150].T, usevlines=True, maxlags=50, normed=True)
ax1.grid(True)

ax2.acorr(reservoirStatesBuffer1[16,50:150].T, usevlines=True, normed=True, maxlags=50)
ax2.grid(True)

plt.show()

In [ ]:
fig, [ax1, ax2] = plt.subplots(2, 1, sharex=True)
ax1.xcorr(roi_data[7,50:150].T, roi_data[16,50:150].T, usevlines=False, maxlags=50, normed=True)
ax1.grid(True)

ax2.acorr(roi_data[16,50:150].T, usevlines=False, normed=True, maxlags=50)
ax2.grid(True)

plt.show()






In [ ]:
wiki2vec.get_word_vector("aezr")

In [ ]:
word = "qsdf"
if '<Word sangane>' in wiki2vec.dictionary:
    print("great")

In [ ]:
for token in wiki2vec.dictionary:
    print(token)

In [ ]:
if wiki2vec.get_word('qsdfqsdf'):
    print("great")

In [ ]:
###### 